In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

### data

In [2]:
np.random.seed(47)
mnist = input_data.read_data_sets("D:/work/data/Python/tensorflow/mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting D:/work/data/Python/tensorflow/mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting D:/work/data/Python/tensorflow/mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting D:/work/data/Python/tensorflow/mnist/data/t10k-images-idx3-ubyte.gz
Extracting D:/work/data/Python/tensorflow/mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### parameters

In [3]:
learning_rate = 0.1
training_epochs = 10
batch_size = 64

# Architecture
n_hidden_1 = 128
n_input = 784
n_classes = 10

### model

In [5]:
g = tf.Graph()
with g.as_default():
    
    tf.set_random_seed(123)

    # Input data
    tf_x = tf.placeholder(tf.float32, [None, n_input], name='features')
    tf_y = tf.placeholder(tf.float32, [None, n_classes], name='targets')

    # Model parameters
    weights = {
        'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1], stddev=0.1)),
        'out': tf.Variable(tf.truncated_normal([n_hidden_1, n_classes], stddev=0.1))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden_1])),
        'out': tf.Variable(tf.zeros([n_classes]))
    }

    # Forward Propagation
    h1_z = tf.add(tf.matmul(tf_x, weights['h1']), biases['b1'])
    h1_act = tf.nn.sigmoid(h1_z)
    out_z = tf.matmul(h1_act, weights['out']) + biases['out']
    out_act = tf.nn.softmax(out_z, name='predicted_probabilities')
    out_labels = tf.argmax(out_z, axis=1, name='predicted_labels')
    
    ######################
    # Forward Propagation
    ######################

    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_y, logits=out_z)
    cost = tf.reduce_mean(loss, name='cost')
    
    ##################
    # Backpropagation
    ##################

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train = optimizer.minimize(cost, name='train')

    ##############
    # Prediction
    ##############

    correct_prediction = tf.equal(tf.argmax(tf_y, 1), out_labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

### train

In [6]:
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = mnist.train.num_examples // batch_size

        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c = sess.run(['train', 'cost:0'], feed_dict={'features:0': batch_x,
                                                            'targets:0': batch_y})
            avg_cost += c
        
        train_acc = sess.run('accuracy:0', feed_dict={'features:0': mnist.train.images,
                                                      'targets:0': mnist.train.labels})
        valid_acc = sess.run('accuracy:0', feed_dict={'features:0': mnist.validation.images,
                                                      'targets:0': mnist.validation.labels})  
        
        print("Epoch: %03d | AvgCost: %.3f" % (epoch + 1, avg_cost / (i + 1)), end="")
        print(" | Train/Valid ACC: %.3f/%.3f" % (train_acc, valid_acc))
        
    test_acc = sess.run(accuracy, feed_dict={'features:0': mnist.test.images,
                                             'targets:0': mnist.test.labels})
    print('Test ACC: %.3f' % test_acc)

Epoch: 001 | AvgCost: 0.786 | Train/Valid ACC: 0.888/0.894
Epoch: 002 | AvgCost: 0.370 | Train/Valid ACC: 0.907/0.911
Epoch: 003 | AvgCost: 0.317 | Train/Valid ACC: 0.914/0.919
Epoch: 004 | AvgCost: 0.288 | Train/Valid ACC: 0.922/0.924
Epoch: 005 | AvgCost: 0.268 | Train/Valid ACC: 0.927/0.929
Epoch: 006 | AvgCost: 0.250 | Train/Valid ACC: 0.933/0.935
Epoch: 007 | AvgCost: 0.235 | Train/Valid ACC: 0.936/0.938
Epoch: 008 | AvgCost: 0.222 | Train/Valid ACC: 0.939/0.939
Epoch: 009 | AvgCost: 0.209 | Train/Valid ACC: 0.943/0.943
Epoch: 010 | AvgCost: 0.198 | Train/Valid ACC: 0.946/0.947
Test ACC: 0.944


## 2. Using tf.gradients (low level)

### model

In [7]:
g = tf.Graph()
with g.as_default():
    
    tf.set_random_seed(123)

    # Input data
    tf_x = tf.placeholder(tf.float32, [None, n_input], name='features')
    tf_y = tf.placeholder(tf.float32, [None, n_classes], name='targets')

    # Model parameters
    weights = {
        'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1], stddev=0.1)),
        'out': tf.Variable(tf.truncated_normal([n_hidden_1, n_classes], stddev=0.1))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden_1])),
        'out': tf.Variable(tf.zeros([n_classes]))
    }

    ######################
    # Forward Propagation
    ######################

    h1_z = tf.add(tf.matmul(tf_x, weights['h1']), biases['b1'])
    h1_act = tf.nn.sigmoid(h1_z)
    out_z = tf.matmul(h1_act, weights['out']) + biases['out']
    out_act = tf.nn.softmax(out_z, name='predicted_probabilities')
    out_labels = tf.argmax(out_z, axis=1, name='predicted_labels')
    
    # Loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=out_z, labels=tf_y)
    cost = tf.reduce_mean(loss, name='cost')
    
    ##################
    # Backpropagation
    ##################

    # Get Gradients
    dc_dw_out, dc_db_out = tf.gradients(cost, [weights['out'], biases['out']])
    dc_dw_1, dc_db_1 = tf.gradients(cost, [weights['h1'], biases['b1']])
    
    # Update Weights
    upd_w_1 = tf.assign(weights['h1'], weights['h1'] - learning_rate * dc_dw_1)
    upd_b_1 = tf.assign(biases['b1'], biases['b1'] - learning_rate * dc_db_1)
    upd_w_out = tf.assign(weights['out'], weights['out'] - learning_rate * dc_dw_out)
    upd_b_out = tf.assign(biases['out'], biases['out'] - learning_rate * dc_db_out)
    
    train = tf.group(upd_w_1, upd_b_1, upd_w_out, upd_b_out, name='train')

    ##############
    # Prediction
    ##############

    correct_prediction = tf.equal(tf.argmax(tf_y, 1), out_labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

### train

In [8]:
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = mnist.train.num_examples // batch_size

        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c = sess.run(['train', 'cost:0'], feed_dict={'features:0': batch_x,
                                                            'targets:0': batch_y})
            avg_cost += c
        
        train_acc = sess.run('accuracy:0', feed_dict={'features:0': mnist.train.images,
                                                      'targets:0': mnist.train.labels})
        valid_acc = sess.run('accuracy:0', feed_dict={'features:0': mnist.validation.images,
                                                      'targets:0': mnist.validation.labels})  
        
        print("Epoch: %03d | AvgCost: %.3f" % (epoch + 1, avg_cost / (i + 1)), end="")
        print(" | Train/Valid ACC: %.3f/%.3f" % (train_acc, valid_acc))
        
    test_acc = sess.run(accuracy, feed_dict={'features:0': mnist.test.images,
                                             'targets:0': mnist.test.labels})
    print('Test ACC: %.3f' % test_acc)

Epoch: 001 | AvgCost: 0.785 | Train/Valid ACC: 0.886/0.893
Epoch: 002 | AvgCost: 0.370 | Train/Valid ACC: 0.906/0.911
Epoch: 003 | AvgCost: 0.317 | Train/Valid ACC: 0.915/0.919
Epoch: 004 | AvgCost: 0.289 | Train/Valid ACC: 0.921/0.926
Epoch: 005 | AvgCost: 0.268 | Train/Valid ACC: 0.928/0.930
Epoch: 006 | AvgCost: 0.250 | Train/Valid ACC: 0.932/0.934
Epoch: 007 | AvgCost: 0.235 | Train/Valid ACC: 0.937/0.938
Epoch: 008 | AvgCost: 0.221 | Train/Valid ACC: 0.940/0.940
Epoch: 009 | AvgCost: 0.209 | Train/Valid ACC: 0.943/0.944
Epoch: 010 | AvgCost: 0.199 | Train/Valid ACC: 0.947/0.948
Test ACC: 0.945


## 3. Gradient Descent from scratch (very low level)

In [9]:
g = tf.Graph()
with g.as_default():
    
    tf.set_random_seed(123)

    # Input data
    tf_x = tf.placeholder(tf.float32, [None, n_input], name='features')
    tf_y = tf.placeholder(tf.float32, [None, n_classes], name='targets')

    # Model parameters
    weights = {
        'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1], stddev=0.1)),
        'out': tf.Variable(tf.truncated_normal([n_hidden_1, n_classes], stddev=0.1))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden_1])),
        'out': tf.Variable(tf.zeros([n_classes]))
    }

    ######################
    # Forward Propagation
    ######################
    
    h1_z = tf.add(tf.matmul(tf_x, weights['h1']), biases['b1'])
    h1_act = tf.nn.sigmoid(h1_z)
    out_z = tf.matmul(h1_act, weights['out']) + biases['out']
    out_act = tf.nn.softmax(out_z, name='predicted_probabilities')
    out_labels = tf.argmax(out_z, axis=1, name='predicted_labels')
    
    # Loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=out_z, labels=tf_y)
    cost = tf.reduce_mean(loss, name='cost')
    
    ##################
    # Backpropagation
    ##################
    
    # Get Gradients
    
    # input/output dim: [n_samples, n_classlabels]
    sigma_out = (out_act - tf_y) / batch_size
    
    # input/output dim: [n_samples, n_hidden_1]
    softmax_derivative_h1 = h1_act * (1. - h1_act)
    
    # input dim: [n_samples, n_classlabels] dot [n_classlabels, n_hidden]
    # output dim: [n_samples, n_hidden]
    sigma_h = (tf.matmul(sigma_out, tf.transpose(weights['out'])) *
               softmax_derivative_h1)
   
    # input dim: [n_features, n_samples] dot [n_samples, n_hidden]
    # output dim: [n_features, n_hidden]
    grad_w_h1 = tf.matmul(tf.transpose(tf_x), sigma_h)
    grad_b_h1 = tf.reduce_sum(sigma_h, axis=0)

    # input dim: [n_hidden, n_samples] dot [n_samples, n_classlabels]
    # output dim: [n_hidden, n_classlabels]
    grad_w_out = tf.matmul(tf.transpose(h1_act), sigma_out)
    grad_b_out = tf.reduce_sum(sigma_out, axis=0)
    
    # Update weights
    upd_w_1 = tf.assign(weights['h1'], weights['h1'] - learning_rate * grad_w_h1)
    upd_b_1 = tf.assign(biases['b1'], biases['b1'] - learning_rate * grad_b_h1)
    upd_w_out = tf.assign(weights['out'], weights['out'] - learning_rate * grad_w_out)
    upd_b_out = tf.assign(biases['out'], biases['out'] - learning_rate * grad_b_out)
    
    train = tf.group(upd_w_1, upd_b_1, upd_w_out, upd_b_out, name='train')
    
    ##############
    # Prediction
    ##############

    correct_prediction = tf.equal(tf.argmax(tf_y, 1), out_labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

### train

In [10]:
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = mnist.train.num_examples // batch_size

        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c = sess.run(['train', 'cost:0'], feed_dict={'features:0': batch_x,
                                                            'targets:0': batch_y})
            avg_cost += c
        
        train_acc = sess.run('accuracy:0', feed_dict={'features:0': mnist.train.images,
                                                      'targets:0': mnist.train.labels})
        valid_acc = sess.run('accuracy:0', feed_dict={'features:0': mnist.validation.images,
                                                      'targets:0': mnist.validation.labels})  
        
        print("Epoch: %03d | AvgCost: %.3f" % (epoch + 1, avg_cost / (i + 1)), end="")
        print(" | Train/Valid ACC: %.3f/%.3f" % (train_acc, valid_acc))
        
    test_acc = sess.run(accuracy, feed_dict={'features:0': mnist.test.images,
                                             'targets:0': mnist.test.labels})
    print('Test ACC: %.3f' % test_acc)

Epoch: 001 | AvgCost: 0.784 | Train/Valid ACC: 0.887/0.891
Epoch: 002 | AvgCost: 0.370 | Train/Valid ACC: 0.904/0.913
Epoch: 003 | AvgCost: 0.317 | Train/Valid ACC: 0.915/0.919
Epoch: 004 | AvgCost: 0.288 | Train/Valid ACC: 0.921/0.929
Epoch: 005 | AvgCost: 0.268 | Train/Valid ACC: 0.928/0.933
Epoch: 006 | AvgCost: 0.250 | Train/Valid ACC: 0.932/0.935
Epoch: 007 | AvgCost: 0.235 | Train/Valid ACC: 0.936/0.939
Epoch: 008 | AvgCost: 0.221 | Train/Valid ACC: 0.940/0.941
Epoch: 009 | AvgCost: 0.210 | Train/Valid ACC: 0.944/0.945
Epoch: 010 | AvgCost: 0.198 | Train/Valid ACC: 0.946/0.949
Test ACC: 0.945
